In [1]:
import pandas as pd
import numpy as np

In [3]:
class Tree:
    def type_of_cols(sf,data):
        # if the availabe no of unique values of a column is 
        #greater than 5 then it can be classified as continues
        #else as categorial
        col_type_and_name=[]
        col_type=[]
        no_of_unique=5
        for col in data.columns[:-1]:
            no_of_col_unique= data[col].nunique()
            if data[col].dtypes==object or no_of_col_unique<no_of_unique:
                col_type_and_name.append((col,"categorical"))
                col_type.append("categorical")
            else:
                col_type_and_name.append((col,"continuous"))
                col_type.append("continuous")
        return col_type
    def get_splits(sf,data):
        splits = {} # dict to store the possible splits of our data based on either categorial or continues data
        feature_type=sf.type_of_cols(data) # get the type of data of each column of feature set
        for column_index in range(data.shape[1]-1):          # excluding the last column which is the label
            values = data.iloc[:, column_index] # accessing the all the column value based on index nor the column name
            unique_values = np.unique(values) #fetch the unique values of the respective column
            type_of_feature = feature_type[column_index] # get data type of column i.e categorial or continues
            if type_of_feature == "continuous":
                splits[column_index] = []
                for index in range(len(unique_values)):
                    if index != 0:
                        current_value = unique_values[index]
                        previous_value = unique_values[index - 1]
                        split = (current_value + previous_value) / 2
                        splits[column_index].append(split)
            # feature is categorical(there need to be at least 2 unique values, otherwise in the 
            #split_data function data_below would contain all data points and data_above would be empty)
            elif len(unique_values) > 1:
                splits[column_index] = unique_values
        return splits
    def split_data(sf,data, column, value):
        #based on to column's data type we will destribute the data into two portion's 
        feature_type=sf.type_of_cols(data)
        split_values = data.iloc[:,column]
        type_of_feature = feature_type[column]
        if type_of_feature == "continuous": # in case of continues data we will use greater or lesser than operator
            left = data[split_values <= value]
            right = data[split_values >  value]
        else:# in case of categorial data will use logical equal or not equal operator
            left = data[split_values == value]
            right = data[split_values != value]
        return left,right

    def entropy(sf,data):#we are calculating the entropy of class 
        prob=list(dict(data.iloc[:, -1].value_counts(normalize=True)).values())
        entropy = sum(prob* -np.log2(prob))    
        return entropy

    def entropy_data(sf,left,right):    
        n = len(left) + len(right)
        p_left = len(left) / n
        p_right = len(right) / n
        entropy_ =  (p_left * sf.entropy(left)+ p_right *sf.entropy(right))
        return entropy_
    
    def best_split(sf,data,splits):
        entropy = 9999 #we assume a dummy entropy to compare with obtained entropy for the first time 
        for col in splits: # iterating over the splits obtained by the get_split method for each columns
            for val in splits[col]: # iterating over the splits of a indivisual columns
                left, right = sf.split_data(data, column=col, value=val)# spliting the data according to obtained split(val) of a column
                current_entropy = sf.entropy_data(left,right) #calculating the entropy of a column
                if current_entropy <= entropy: # if obtained entropy is lesser than assumed entropy 
                    entropy = current_entropy #then assume the obtained entropy as best entropy
                    best_column = col # the current column can be termend as best column and the split too
                    best_split = val
        return best_column, best_split
    
    def tree_builder(sf,df, counter=0, min_samples=2, max_depth=5):
        from collections import Counter
        # data preparations
        column=df.columns #store the column name
        feature_type=sf.type_of_cols(df) # store the column value type i.e categorial/continues 
        data = df
        # base cases
        if (df.iloc[:,-1].nunique()==1) or (len(data) < min_samples) or (counter == max_depth):
            classes= Counter(data.iloc[:,-1]).most_common(1)[0][0]
            return classes
        # recursive part
        else:    
            counter += 1
            splits = sf.get_splits(data)#calculating the splits of each columns
            split_column, split_value = sf.best_split(data,splits) #getting the best column and split value
            left, right = sf.split_data(data, split_column, split_value) # based on the above split and column divide the data 
            # determine question
            feature_name = column[split_column] #pick the column name
            type_of_feature = feature_type[split_column]
            if type_of_feature == "continuous":
                question = "{} <= {}".format(feature_name, split_value) # assign the column name ,type of operator to be used and the split value            
            # feature is categorical
            else:
                question = "{} = {}".format(feature_name, split_value)
            # instantiate sub-tree
            mytree = {question: []}
            # find answers (recursion)
            ans_yes = sf.tree_builder(left, counter, min_samples, max_depth) # left leave is for yes where tree traversal stops
            ans_no = sf.tree_builder(right, counter, min_samples, max_depth) # right leave needs few more nodes
            # If the answers are the same, then there is no point in asking the question.
            # This could happen when the data is classified even though it is not pure
            # yet (min_samples or max_depth base case).
            if ans_yes == ans_no:
                mytree = ans_yes
            else:
                mytree[question].append(ans_yes)
                mytree[question].append(ans_no)
        
            return mytree

In [4]:
data=pd.read_csv('bcancer.csv')
data.drop(["Unnamed: 0",'a'],axis=1,inplace=True)
train=data[:int(len(data)*0.8)]
test=data[int(len(data)*0.8):]
algo=Tree()
#algo.best_split(train,algo.get_splits(train)),algo.entropy_data(*algo.split_data(train,2,2.5))
mytree=algo.tree_builder(train)
mytree

{'d <= 2.5': [{'b <= 5.5': [{'g <= 4.5': [2, {'e = 3': [4, 2]}]},
    {'g <= 1.5': [2, 4]}]},
  {'c <= 1.5': [{'i = 10': [4, 2]},
    {'b <= 6.5': [{'g <= 6.0': [{'i <= 2.5': [2, 4]}, {'g <= 11.0': [4, 2]}]},
      4]}]}]}

In [5]:
mytree['d <= 2.5'][1]

{'c <= 1.5': [{'i = 10': [4, 2]},
  {'b <= 6.5': [{'g <= 6.0': [{'i <= 2.5': [2, 4]}, {'g <= 11.0': [4, 2]}]},
    4]}]}

In [6]:
class DecisionTree(Tree):
    def fit(sf,train):
        sf.tree=sf.tree_builder(train)
        return sf.tree
    def predict(sf,dx,tree):
        root_node = list(tree.keys())[0] # fetch the root node's value i.e our dict keys which consits of column name,operator and split value 
        column,operator,split=root_node.split(" ") # we have used the space as a seprator b/w the three data
        if operator == "<=": # if the operator is lesser than or equal it means th column type is continues
            
            if dx[column] <= float(split):
                result = tree[root_node][0]
            else:
                result = tree[root_node][1]
        # if column  is categorical then we can use logical equal operator
        else:
            if str(dx[column]) == split:
                result = tree[root_node][0]
            else:
                result = tree[root_node][1]
        if type(result)!=dict: # if the result is dict then we have more nodes to be traversed
            return result
        # else recursively travers the entire tree for accurate results
        else:
            return sf.predict(dx,result)
    def score(sf,x,y):
        c=0 # we will pass a indivisual x into predict and compare the given y with the predicted y if both are equal then c will be incremented by 1 
        for i in range(len(x)):
            if sf.predict(x.iloc[i],sf.tree)==y.iloc[i]:
                c+=1
        return c/len(x) # finally resultant is correct prediction by the total len of x

In [8]:
myalgo=DecisionTree()
myalgo.fit(train)
myalgo.score(test.drop('class',axis=1),test['class'])

0.9785714285714285